# 模型构造
让我们回顾⼀下在[“多层感知机的简洁实现”](../dlbasic/3.10mlp-gluon.ipynb) ⼀节中含单隐藏层的多层感知机的实现⽅法。我们
⾸先构造Sequential实例，然后依次添加两个全连接层。其中第⼀层的输出⼤小为256，即隐
藏层单元个数是256；第⼆层的输出⼤小为10，即输出层单元个数是10。我们在上⼀章的其他节
中也使⽤了Sequential类构造模型。这⾥我们介绍另外⼀种基于Block类的模型构造⽅法：它
让模型构造更加灵活。
## 继承Block类来构造模型
Block类是nn模块⾥提供的⼀个模型构造类，我们可以继承它来定义我们想要的模型。下⾯继
承Block类构造本节开头提到的多层感知机。这⾥定义的MLP类重载了Block类的__init__函
数和forward函数。它们分别⽤于创建模型参数和定义前向计算。前向计算也即正向传播

In [9]:
from mxnet import nd
from mxnet.gluon import nn
class MLP(nn.Block):
    # 声明带有模型参数的层，这⾥声明了两个全连接层
    def __init__(self, **kwargs):
        # 调⽤MLP⽗类Block的构造函数来进⾏必要的初始化。这样在构造实例时还可以指定其他函数
        # 参数，如“模型参数的访问、初始化和共享”⼀节将介绍的模型参数params
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Dense(256, activation='relu')
        self.output = nn.Dense(10)
    # 定义模型的前向计算，即如何根据输⼊x计算返回所需要的模型输出
    def forward(self, x):
        return self.output(self.hidden(x))

以上的MLP类中⽆须定义反向传播函数。系统将通过⾃动求梯度而⾃动⽣成反向传播所需
的backward函数。我们可以实例化MLP类得到模型变量net。下⾯的代码初始化net并传⼊
输⼊数据X做⼀次前向计算。其中， net(X)会调⽤MLP继承⾃Block类的__call__函数，
这个函数将调⽤MLP类定义的forward函数来完成前向计算。

In [6]:
X = nd.random.normal(shape=(2,20))
net = MLP()
net.initialize()
net(X)


[[-0.00884219 -0.01361075  0.00989831 -0.05785033 -0.09113112 -0.03276934
   0.10951708 -0.0594931  -0.03217464 -0.03013538]
 [ 0.01740783  0.01566995 -0.12216819 -0.00195891 -0.00130794 -0.0514056
   0.0844209   0.00769345  0.02565179 -0.02311425]]
<NDArray 2x10 @cpu(0)>

注意，这⾥并没有将Block类命名为Layer（层）或者Model（模型）之类的名字，这是因为该
类是⼀个可供⾃由组建的部件。它的⼦类既可以是⼀个层（如Gluon提供的Dense类），⼜可以是
⼀个模型（如这⾥定义的MLP类），或者是模型的⼀个部分。我们下⾯通过两个例⼦来展⽰它的灵
活性。
## Sequential类继承自Block
我们刚刚提到， Block类是⼀个通⽤的部件。事实上， Sequential类继承⾃Block类。当模型的前
向计算为简单串联各个层的计算时，可以通过更加简单的⽅式定义模型。这正是Sequential类
的⽬的：它提供add函数来逐⼀添加串联的Block⼦类实例，而模型的前向计算就是将这些实例
按添加的顺序逐⼀计算。
下⾯我们实现⼀个与Sequential类有相同功能的MySequential类。这或许可以帮助读者更
加清晰地理解Sequential类的⼯作机制。

In [12]:
class MySequential(nn.Block):
    def __init__(self, **kwargs):
        super(MySequential, self).__init__(**kwargs)
    def add(self, block):
        # block是⼀个Block⼦类实例，假设它有⼀个独⼀⽆⼆的名字。我们将它保存在Block类的
        # 成员变量_children⾥，其类型是OrderedDict。当MySequential实例调⽤
        # initialize函数时，系统会⾃动对_children⾥所有成员初始化
        self._children[block.name] = block
    def forward(self, x):
        # OrderedDict保证会按照成员添加时的顺序遍历成员
        for block in self._children.values():
            x = block(x)
        return x

我们⽤MySequential类来实现前⾯描述的MLP类，并使⽤随机初始化的模型做⼀次前向计算。

In [13]:
net = MySequential()
net.add(nn.Dense(256,activation='relu'))
net.add(nn.Dense(10))
net.initialize()
net(X)


[[-0.05044801  0.08025599  0.04171951 -0.02426371  0.06428756 -0.00904521
   0.08975367 -0.05393827  0.01967067  0.12805972]
 [-0.00394633  0.0259387   0.02448444 -0.18089381  0.23608395 -0.0582966
  -0.01937858 -0.04208054  0.09123673 -0.05099846]]
<NDArray 2x10 @cpu(0)>

可以观察到这⾥MySequential类的使⽤跟[“多层感知机的简洁实现”](../dlbasic/3.10mlp-gluon.ipynb) ⼀节中Sequential类的
使⽤没什么区别。
## 构造复杂的模型
虽然Sequential类可以使模型构造更加简单，且不需要定义forward函数，但直接继承Block类
可以极⼤地拓展模型构造的灵活性。下⾯我们构造⼀个稍微复杂点的⽹络FancyMLP。在这个⽹
络中，我们通过get_constant函数创建训练中不被迭代的参数，即常数参数。在前向计算中，
除了使⽤创建的常数参数外，我们还使⽤NDArray的函数和Python的控制流，并多次调⽤相同的层。

In [14]:
class FancyMLP(nn.Block):
	def __init__(self, **kwargs):
		super(FancyMLP, self).__init__(**kwargs)
		# 使用get_constant创建的随机权重参数不会在训练中被迭代（即常数参数）
		self.rand_weight = self.params.get_constant(
			'rand_weight', nd.random.uniform(shape=(20, 20)))
		self.dense = nn.Dense(20, activation='relu')

	def forward(self, x):
		x = self.dense(x)
		# 使用创建的常数参数，以及NDArray的relu函数和dot函数
		x = nd.relu(nd.dot(x, self.rand_weight.data()) + 1)
		# 复用全连接层。等价于两个全连接层共享参数
		x = self.dense(x)
		# 控制流，这里我们需要调用asscalar函数来返回标量进行比较
		while x.norm().asscalar() > 1:
			x /= 2
		if x.norm().asscalar() < 0.8:
			x *= 10
		return x.sum()

在这个FancyMLP模型中，我们使⽤了常数权重rand_weight（注意它不是模型参数）、做了矩
阵乘法操作（nd.dot）并重复使⽤了相同的Dense层。下⾯我们来测试该模型的随机初始化和
前向计算。

In [15]:
net = FancyMLP()
net.initialize()
net(X)


[24.901281]
<NDArray 1 @cpu(0)>

因为FancyMLP和Sequential类都是Block类的⼦类，所以我们可以嵌套调⽤它们

In [16]:
class NestMLP(nn.Block):
	def __init__(self, **kwargs):
		super(NestMLP, self).__init__(**kwargs)
		self.net = nn.Sequential()
		self.net.add(nn.Dense(64, activation='relu'),
					 nn.Dense(32, activation='relu'))
		self.dense = nn.Dense(16, activation='relu')

	def forward(self, x):
		return self.dense(self.net(x))

net = nn.Sequential()
net.add(NestMLP(), nn.Dense(20), FancyMLP())

net.initialize()
net(X)


[4.0478725]
<NDArray 1 @cpu(0)>

## 小结
- 可以通过继承Block类来构造模型。
- Sequential类继承⾃Block类。
- 虽然Sequential类可以使模型构造更加简单，但直接继承Block类可以极⼤地拓展模型构造的灵活性。